In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [13]:
# reading in csvs
intakes_df = pd.read_csv('../cleanedSources/intakeCleaned.csv')
outcomes_df = pd.read_csv('../cleanedSources/outcomesCleaned.csv')

In [21]:
# examining the df
intakes_df.head()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age,breed,colour,age_intake_months,age_intake_years
0,A786884,Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,24.0,2.00
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,96.0,8.00
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,11.0,0.92
3,A665644,Unknown,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,1.0,0.00
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,48.0,4.00


In [24]:
# examining the df
outcomes_df.head()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age,breed,colour,age_outcome_months,age_outcome_years
0,A794011,Chunk,2019-05-08 18:20:00,2017-05-02 00:00:00,Rto-Adopt,Unspecified,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,24.0,2.00
1,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12 00:00:00,Adoption,Unspecified,Dog,Neutered Male,8 years,Chihuahua Shorthair Mix,White/Brown,12.0,1.00
2,A821648,Unknown,2020-08-16 11:38:00,2019-08-16 00:00:00,Euthanasia,Unspecified,Other,Unknown,11 months,Raccoon,Gray,12.0,1.00
3,A720371,Moose,2016-02-13 17:59:00,2015-10-08 00:00:00,Adoption,Unspecified,Dog,Neutered Male,4 weeks,Anatol Shepherd/Labrador Retriever,Buff,4.0,0.33
4,A674754,Unknown,2014-03-18 11:47:00,2014-03-12 00:00:00,Transfer,Partner,Cat,Intact Male,4 years,Domestic Shorthair Mix,Orange Tabby,0.0,0.00


In [25]:
# function to remove negative values in the age cols
def neg_fix(row):
    if row['age'][0] == '-':
        val = row['age'][1:]
    else:
            val = row['age']
    return val

In [29]:
# function to return animal ages in months and years, as ints
def norm_age(row):
    # if it ends with year/years, multiply by 12 and enter into month col; enter number into year col
    if row['age'].split(' ')[1] in ['year', 'years']:
        age_val = int(row['age'].split(' ')[0])
        age_months = round(age_val * 12, 2)
        age_years = round(age_val, 2)
    # if it ends with month/months, enter number into month col; divide by 12 and place into year col
    elif row['age'].split(' ')[1] in ['month', 'months']:
        age_val = int(row['age'].split(' ')[0])
        age_months = round(age_val, 2)
        age_years = round(age_val/12, 2)
    # if it ends with week/weeks, divide by four and place into month col; leave default value in year col
    elif row['age'].split(' ')[1] in ['week', 'weeks']:
        age_val = int(row['age'].split(' ')[0])
        age_months = round(age_val/4, 2)
        age_years = 0
    # else (if it ends with day or days), leave default value in both cols
    else:
        age_months = 0
        age_years = 0
    return np.array((age_months, age_years))

In [26]:
# removing negative vals from both dfs
intakes_df['age'] = intakes_df.apply(neg_fix, axis=1)
outcomes_df['age'] = intakes_df.apply(neg_fix, axis=1)

In [27]:
intakes_df.head()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age,breed,colour,age_intake_months,age_intake_years
0,A786884,Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,24.0,2.00
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,96.0,8.00
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,11.0,0.92
3,A665644,Unknown,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,1.0,0.00
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,48.0,4.00


In [28]:
outcomes_df.head()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age,breed,colour,age_outcome_months,age_outcome_years
0,A794011,Chunk,2019-05-08 18:20:00,2017-05-02 00:00:00,Rto-Adopt,Unspecified,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,24.0,2.00
1,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12 00:00:00,Adoption,Unspecified,Dog,Neutered Male,8 years,Chihuahua Shorthair Mix,White/Brown,12.0,1.00
2,A821648,Unknown,2020-08-16 11:38:00,2019-08-16 00:00:00,Euthanasia,Unspecified,Other,Unknown,11 months,Raccoon,Gray,12.0,1.00
3,A720371,Moose,2016-02-13 17:59:00,2015-10-08 00:00:00,Adoption,Unspecified,Dog,Neutered Male,4 weeks,Anatol Shepherd/Labrador Retriever,Buff,4.0,0.33
4,A674754,Unknown,2014-03-18 11:47:00,2014-03-12 00:00:00,Transfer,Partner,Cat,Intact Male,4 years,Domestic Shorthair Mix,Orange Tabby,0.0,0.00


In [32]:
# applying the norm_age function to both dfs, creating age_months, age_years cols for both
intakes_df['age_intake_months'], intakes_df['age_intake_years'] = intakes_df.apply(norm_age, axis=1, result_type='expand').T.values
outcomes_df['age_outcome_months'], outcomes_df['age_outcome_years'] = outcomes_df.apply(norm_age, axis=1, result_type='expand').T.values

In [39]:
# relabelling the base-age cols to correspond to whether intake/outcome
intakes_df.columns = ['id', 'name_intake', 'dateTime_intake', 'loc_found', 'intake_type',
       'intake_condition', 'animal_type', 'sex_intake', 'age_intake', 'breed',
       'colour', 'age_intake_months', 'age_intake_years']
outcomes_df.columns = ['id', 'name_outcome', 'dateTime_outcome', 'dob', 'outcome_type',
       'outcome_subtype', 'animal_type', 'sex_outcome', 'age_outcome', 'breed',
       'colour', 'age_outcome_months', 'age_outcome_years']

In [37]:
intakes_df.head()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age_intake,breed,colour,age_intake_months,age_intake_years
0,A786884,Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,24.0,2.00
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,96.0,8.00
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,11.0,0.92
3,A665644,Unknown,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,1.0,0.00
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,48.0,4.00


In [40]:
outcomes_df.head()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age_outcome,breed,colour,age_outcome_months,age_outcome_years
0,A794011,Chunk,2019-05-08 18:20:00,2017-05-02 00:00:00,Rto-Adopt,Unspecified,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,24.0,2.00
1,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12 00:00:00,Adoption,Unspecified,Dog,Neutered Male,8 years,Chihuahua Shorthair Mix,White/Brown,96.0,8.00
2,A821648,Unknown,2020-08-16 11:38:00,2019-08-16 00:00:00,Euthanasia,Unspecified,Other,Unknown,11 months,Raccoon,Gray,11.0,0.92
3,A720371,Moose,2016-02-13 17:59:00,2015-10-08 00:00:00,Adoption,Unspecified,Dog,Neutered Male,4 weeks,Anatol Shepherd/Labrador Retriever,Buff,1.0,0.00
4,A674754,Unknown,2014-03-18 11:47:00,2014-03-12 00:00:00,Transfer,Partner,Cat,Intact Male,4 years,Domestic Shorthair Mix,Orange Tabby,48.0,4.00


In [41]:
# re-saving to csvs
intakes_df.to_csv('../cleanedSources/intakeCleaned.csv', index=False)
outcomes_df.to_csv('../cleanedSources/outcomesCleaned.csv', index=False)